In [1]:
#Concept taken from https://www.youtube.com/watch?v=HXDD7-EnGBY

import cv2

In [2]:
classnames = []

with open("coco.labels","rt") as fl:
    classnames = fl.read().rstrip("\n").split("\n");

configPath = "ssd_coco.pbtxt"
weightFile = "frozen_inference_graph.pb"

net = cv2.dnn_DetectionModel(weightFile, configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

THRESHOLD = 0.7 #Change it to choose highly confident objects

while True:
    
    isAvail, img = cap.read() #isAvail is status or frame read
    if not isAvail:
        continue
    
    classIds, confs, bbox = net.detect(img, confThreshold = THRESHOLD)

    if len(classIds) == 0 or len(confs) == 0:
        continue
        
    for classId, confidence, boxx in zip(classIds.flatten(), confs.flatten(), bbox):
        cv2.rectangle(img, boxx, color = (255,0,0), thickness = 4)
        cv2.putText(img, str(classnames[classId-1])+" "+str(round(confidence*100,1)), (boxx[0], boxx[1]-10), cv2.FONT_HERSHEY_SIMPLEX,1, (255,0,0),2)
        cv2.imshow("Objects", img)
    if cv2.waitKey(1) & 0XFF == 27: #Press ESC
        break

cap.release()
cv2.destroyAllWindows()